In [1]:
import os

In [2]:
%pwd

'c:\\Users\\PMLS\\Desktop\\Jupyter notebook\\Campusx Codes\\Deep-Learning-Project\\cnnclassifier\\notebooks'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\PMLS\\Desktop\\Jupyter notebook\\Campusx Codes\\Deep-Learning-Project\\cnnclassifier'

## 1. Update config.yaml

```yaml
model_artifacts_root: models

prepare_base_model:
  root_dir: models/prepare_base_model
  base_model_path: models/prepare_base_model/base_model.h5
  updated_base_model_path: models/prepare_base_model/base_model_updated.h5
```

## 3. Update params.yaml

```yaml
AUGMENTATION: True
IMAGE_SIZE: [224, 224, 3] # as per VGG16 model
BATCH_SIZE: 32
INCLUDE_TOP: False
EPOCHS: 1
CLASSES: 2
WEIGHTS: imagenet
LEARNING_RATE: 0.01
ACTIVATION: 'softmax'
FREEZE_ALL: True
FREEZE_TILL: None
```

## 3.1 Update src/constant/\_\_init\_\_.py

## 4. Update the entity

In [1]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_augmentation: bool
    params_image_size: list
    params_batch_size: int
    params_include_top: bool
    params_epochs: int
    params_classes: int
    params_weights: str
    params_learning_rate: float
    params_activation: str
    params_freeze_all: bool
    params_freeze_till: int

## 5. Update the configuration manager src/config/configuration.py

In [ ]:
from src.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from src.utils.common import read_yaml, create_directories
from src.logger import CustomLogger

class ConfigurationManager:
    def __init__(
        self,logger: CustomLogger,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.logger = logger
        self.config = read_yaml(config_filepath, logger=self.logger)
        self.params = read_yaml(params_filepath, logger=self.logger)    

        create_directories([self.config.model_artifacts_root], logger=self.logger)
        

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir], logger=self.logger)
        
        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir = Path(config.root_dir),
            base_model_path = Path(config.base_model_path),
            updated_base_model_path = Path(config.updated_base_model_path),
            
            params_augmentation = self.params.AUGMENTATION,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE,
            params_include_top = self.params.INCLUDE_TOP,
            params_epochs = self.params.EPOCHS,
            params_classes = self.params.CLASSES,
            params_weights = self.params.WEIGHTS,
            params_learning_rate = self.params.LEARNING_RATE,
            params_activation = self.params.ACTIVATION,
            params_freeze_all= self.params.FREEZE_ALL,
            params_freeze_till = self.params.FREEZE_TILL
        )
        return prepare_base_model_config

## 6. Update the components \[data preprocessing, model training, and so on\]

In [ ]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config
        
    def get_base_model(self, download_base_model_logger: CustomLogger):
        try:
            self.model = tf.keras.applications.vgg16.VGG16(
                include_top=self.config.params_include_top,
                weights=self.config.params_weights,
                input_tensor=None,
                input_shape=self.config.params_image_size,
                pooling=None,
                classes=self.config.params_classes,
                classifier_activation=self.config.params_activation
            )
            self.save_model(path=self.config.base_model_path, model = self.model, logger=download_base_model_logger)
            
            download_base_model_logger.save_logs(msg="Base model downloaded successfully.", log_level="info")
            
        except Exception as e:
            download_base_model_logger.save_logs(msg=f"Error in downloading base model. Error: {e}", log_level="error")
            raise e

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate, activation, logger: CustomLogger):
        if freeze_all:
            for layer in model.layers:
                layer.trainable = False
        elif (freeze_till is not None) and (freeze_till > 0):
            for layer in model.layers[:-freeze_till]:
                layer.trainable = False

        flatten_in = tf.keras.layers.Flatten()(model.output)
        if activation == "softmax":
            prediction = tf.keras.layers.Dense(units=64, activation='relu')(flatten_in)
            prediction = tf.keras.layers.Dense(units=classes, activation=activation)(prediction)
            logger.save_logs(msg=f"Model Architecture constructed successfully with {activation} activation function and {classes} classes.", log_level="info")
            
        elif activation == "sigmoid":
            prediction = tf.keras.layers.Dense(units=64, activation='relu')(flatten_in)
            prediction = tf.keras.layers.Dense(units=1, activation=activation)(flatten_in)
            logger.save_logs(msg=f"Model Architecture constructed successfully with {activation} activation function and {classes} classes.", log_level="info")
        else:
            logger.save_logs(msg=f"Activation function {activation} not supported. Error: Correct the last layer activation function OR last layer neurons.", log_level="error")
            raise ValueError(f"Activation function {activation} not supported. Error: Correct the last layer activation function OR last layer neurons.")


        full_model = tf.keras.models.Model(inputs=model.input, outputs=prediction)

        if activation == "softmax":
            full_model.compile(
                optimizer= "adam", # tf.keras.optimizers.SGD(learning_rate=learning_rate),
                loss= "sparse_categorical_crossentropy" ,  # tf.keras.losses.SparseCategoricalCrossentropy(),
                metrics= ["accuracy"] # [tf.keras.metrics.SparseCategoricalAccuracy()],
            )
            logger.save_logs(msg="Model compiled successfully with optimizer: SGD, loss: SparseCategoricalCrossentropy and metrics: SparseCategoricalAccuracy.", log_level="info")
        elif activation == "sigmoid":
            full_model.compile(
                optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=[tf.keras.metrics.BinaryAccuracy()],
            )
            logger.save_logs(msg="Model compiled successfully with optimizer: SGD, loss: BinaryCrossentropy and metrics: BinaryAccuracy.", log_level="info")
        else:
            logger.save_logs(msg=f"Activation function {activation} not supported. Error: Only choose activation function from 'softmax' or 'sigmoid'.", log_level="error")
            raise ValueError(f"Activation function {activation} not supported. Error: Only choose activation function from 'softmax' or 'sigmoid'.")
        
        full_model.summary(show_trainable=True, expand_nested=True)
        return full_model
    
    
    def update_base_model(self, prepare_full_model_logger: CustomLogger, update_base_model_logger: CustomLogger):
        try:
            self.full_model = self._prepare_full_model(
                model=self.model,
                classes=self.config.params_classes,
                freeze_all=self.config.params_freeze_all,
                freeze_till=self.config.params_freeze_till,
                learning_rate=self.config.params_learning_rate,
                activation=self.config.params_activation,
                logger=prepare_full_model_logger
            )
            
            self.save_model(path=self.config.updated_base_model_path, model = self.full_model, logger=update_base_model_logger)

            update_base_model_logger.save_logs(msg="Base model updated successfully.", log_level="info")
        except Exception as e:
            update_base_model_logger.save_logs(msg=f"Error in updating base model. Error: {e}", log_level="error")
            raise e            
            
            
            
    @staticmethod
    def save_model(path: Path, model:tf.keras.Model, logger: CustomLogger):
        try:
            model.save(path)
            logger.save_logs(msg=f"Model saved at {path} successfully.", log_level="info")
        except Exception as e:
            logger.save_logs(msg=f"Error in saving model at {path}. Error: {e}", log_level="error")
            raise e


## 7. Update the pipeline

In [ ]:
from src.logger import create_log_path, CustomLogger
import logging

# path to save the log files
download_log_file_path = create_log_path("model/download_model")
download_model_logger = CustomLogger(logger_name="download_model", log_filename=download_log_file_path)
download_model_logger.set_log_level(level=logging.INFO)

prepare_model_log_file_path = create_log_path("model/prepare_model")
prepare_full_model_logger = CustomLogger(logger_name="prepare_model", log_filename=prepare_model_log_file_path)
prepare_full_model_logger.set_log_level(level=logging.INFO)

update_model_log_file_path = create_log_path("model/update_model")
update_base_model_logger = CustomLogger(logger_name="update_model", log_filename=update_model_log_file_path)
update_base_model_logger.set_log_level(level=logging.INFO)

model_log_file_path = create_log_path("model/model")
model_logger = CustomLogger(logger_name="model", log_filename=model_log_file_path)
model_logger.set_log_level(level=logging.INFO)

configuration_log_file_path = create_log_path("configuration")
configuration_logger = CustomLogger(logger_name="configuration", log_filename=configuration_log_file_path)
configuration_logger.set_log_level(level=logging.INFO)


try:
    config = ConfigurationManager(configuration_logger)
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model(download_base_model_logger=download_model_logger)
    prepare_base_model.update_base_model(prepare_full_model_logger=prepare_full_model_logger, update_base_model_logger=update_base_model_logger)
    
    model_logger.save_logs(msg="Model pipeline completed successfully.", log_level="info")
except Exception as e:
    model_logger.save_logs(msg=f"Error in model pipeline. Error: {e}", log_level="error")
    raise e

INFO:model:yaml file: config\config.yaml loaded successfully
INFO:model:yaml file: params.yaml loaded successfully
INFO:model:created directory at: models
INFO:model:created directory at: models/prepare_base_model
INFO:download_model:Model saved at models\prepare_base_model\base_model.h5 successfully.
INFO:download_model:Base model downloaded successfully.
INFO:prepare_model:Model Architecture constructed successfully with softmax activation function and 2 classes.
INFO:prepare_model:Model compiled successfully with optimizer: SGD, loss: SparseCategoricalCrossentropy and metrics: SparseCategoricalAccuracy.


Model: "functional_9"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer_10 (InputLayer) │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block1_conv1 (Conv2D)       │ (None, 224, 224, 64)  │      1,792 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block1_conv2 (Conv2D)       │ (None, 224, 224, 64)  │     36,928 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block1_pool (MaxPooling2D)  │ (None, 112, 112, 64)  │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block2_conv1 (Conv2D)       │ (None, 112, 112, 128) │     73,856 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block2_conv2 (Conv2D)       │ (None, 112, 112, 128) │    147,584 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block2_pool (MaxPooling2D)  │ (None, 56, 56, 128)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block3_conv1 (Conv2D)       │ (None, 56, 56, 256)   │    295,168 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block3_conv2 (Conv2D)       │ (None, 56, 56, 256)   │    590,080 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block3_conv3 (Conv2D)       │ (None, 56, 56, 256)   │    590,080 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block3_pool (MaxPooling2D)  │ (None, 28, 28, 256)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block4_conv1 (Conv2D)       │ (None, 28, 28, 512)   │  1,180,160 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block4_conv2 (Conv2D)       │ (None, 28, 28, 512)   │  2,359,808 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block4_conv3 (Conv2D)       │ (None, 28, 28, 512)   │  2,359,808 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block4_pool (MaxPooling2D)  │ (None, 14, 14, 512)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block5_conv1 (Conv2D)       │ (None, 14, 14, 512)   │  2,359,808 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block5_conv2 (Conv2D)       │ (None, 14, 14, 512)   │  2,359,808 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block5_conv3 (Conv2D)       │ (None, 14, 14, 512)   │  2,359,808 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ block5_pool (MaxPooling2D)  │ (None, 7, 7, 512)     │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ flatten_9 (Flatten)         │ (None, 25088)         │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_11 (Dense)            │ (None, 64)            │  1,605,696 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_12 (Dense)            │ (None, 2)             │        130 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 16,320,514 (62.26 MB)

 Trainable params: 1,605,826 (6.13 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

INFO:update_model:Model saved at models\prepare_base_model\base_model_updated.h5 successfully.
INFO:update_model:Base model updated successfully.
INFO:model:Model pipeline completed successfully.


## 8. Update the main.py

In [ ]:
import logging
from src.logger import CustomLogger, create_log_path
from src.pipeline.stage_02_prepare_base_model import PrepareBaseModelTrainingPipeline


pipeline_log_file_path = create_log_path("pipeline")
pipeline_logger = CustomLogger(
    logger_name="pipeline", log_filename=pipeline_log_file_path
)
pipeline_logger.set_log_level(level=logging.INFO)


configuration_log_file_path = create_log_path("configuration")
configuration_logger = CustomLogger(
    logger_name="configuration", log_filename=configuration_log_file_path
)
configuration_logger.set_log_level(level=logging.INFO)




download_log_file_path = create_log_path("model/download_model")
download_model_logger = CustomLogger(
    logger_name="download_model", log_filename=download_log_file_path
)
download_model_logger.set_log_level(level=logging.INFO)

prepare_model_log_file_path = create_log_path("model/prepare_model")
prepare_full_model_logger = CustomLogger(
    logger_name="prepare_model", log_filename=prepare_model_log_file_path
)
prepare_full_model_logger.set_log_level(level=logging.INFO)

update_model_log_file_path = create_log_path("model/update_model")
update_base_model_logger = CustomLogger(
    logger_name="update_model", log_filename=update_model_log_file_path
)
update_base_model_logger.set_log_level(level=logging.INFO)



STAGE_NAME = "Prepare Base Model stage"

try:
    pipeline_logger.save_logs(
        msg=f">>>>>>>>>>>> stage {STAGE_NAME} started <<<<<<<<<<<<",
        log_level="info",
    )
    obj = PrepareBaseModelTrainingPipeline(
        download_model_logger=download_model_logger,
        prepare_full_model_logger=prepare_full_model_logger,
        update_base_model_logger=update_base_model_logger,
        configuration_logger=configuration_logger,
    )
    obj.main()
    pipeline_logger.save_logs(
        msg=f">>>>>>>>>>>> stage {STAGE_NAME} completed <<<<<<<<<<<<\n\nx============x",
        log_level="info",
    )
except Exception as e:
    pipeline_logger.save_logs(
        msg=f"Error in {STAGE_NAME}. Error: {e}", log_level="error"
    )
    raise e

## 9. Update the dvc.yaml

## 10. app.py